## Scrape AllRecipes for additional features

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import pause

In [3]:
## Get URLs from dataset

recipe=pd.read_csv('/Users/ranaquadri/Documents/predictreciperatings/recipe files/all.csv')

recipe_urls = [item for item in recipes['source_url'] if item.startswith('http://allrecipes')]
len(recipe_urls)


IOError: File /Users/ranaquadri/Documents/predictreciperatings/recipe files/all.csv does not exist

### Step 1: Create a soup object from the home page¶

In [123]:
recipe_info=[]
# Use the requests library to get the html from the home page
# for row in dataframe[column]:
for item in recipe_urls: 
    print item
    current_url = item
    try:
        res = requests.get(item)
    except:
        pause.seconds(5)
        continue
    temp_recipe = {}
    temp_recipe['source_url'] = current_url
    # Create a soup object from the html
    soup = bs(res.content, 'lxml')
    # Object for ratings, descrip, reviews, etc
    try:
        recipe = soup.find('section', {'class': 'recipe-summary clearfix'})
    except:
        pass
    # Find the recipe rating (average not rounded)
    try:
        temp_recipe['rating'] = recipe.find('meta', {'property':'og:rating'})['content']
    except:
        pass
    # Grab the name of the recipe
    try:
        temp_recipe['name'] = recipe.find('h1', {'itemprop': 'name'}).text
    except:
        pass
    # Grab number of reviews
    try:
        temp_recipe['num_reviews'] = recipe.find('meta', {'itemprop': "reviewCount"})['content']
    except:
        pass
    # Grab recipe description
    # The recipe text outputs the string '\r\n before the actual string so you can try different methods to get rid of it
    try:
        temp_recipe['descrip'] = recipe.find('div', {'class': 'submitter__description'}).text
    except:
        pass
    # Grab nutrition info
    try:
        nutrition = soup.find('section', {'itemprop': 'nutrition'})
    except:
        pass
    try:
        temp_recipe['nut_fat'] = nutrition.find('span', {'itemprop': 'fatContent'}).text
    except:
        pass
    try:
        temp_recipe['nut_carb'] = nutrition.find('span', {'itemprop': 'carbohydrateContent'}).text
    except:
        pass
    try:
        temp_recipe['nut_cals'] = nutrition.find('span', {'itemprop': 'calories'}).text.replace(';', '').replace(' calories','')
    except:
        pass
    try:
        temp_recipe['nut_prot'] = nutrition.find('span', {'itemprop': 'proteinContent'}).text
    except:
        pass
    try:
        temp_recipe['nut_choles'] = nutrition.find('span', {'itemprop': 'cholesterolContent'}).text
    except:
        pass
    try:
        temp_recipe['nut_sod'] = nutrition.find('span', {'itemprop': 'sodiumContent'}).text
    except:
        pass
    recipe_info2.append(temp_recipe)
    pause.seconds(20)

## Step 2: Look at scraped data

In [107]:
print len(recipe_info)
print recipe_info[1]

1446
{'rating': '4.69858169555664', 'nut_cals': u'408', 'name': u'Apple Crumb Pie', 'nut_carb': u'69.9', 'nut_prot': u'3.5', 'nut_fat': u'14.6', 'source_url': 'http://allrecipes.com/Recipe/Apple-Crumb-Pie/Detail.aspx', 'num_reviews': '678', 'nut_sod': u'139', 'descrip': u'\r\n"You can add walnuts and raisins to this apple crumb pie to make it even dreamier!"        ', 'nut_choles': u'11'}


In [134]:
# Create a dataframe from scraped data

allrec=pd.DataFrame(recipe_info)
print allrec.head()

                                             descrip  \
0  \r\n"This was my grandmother's apple pie recip...   
1  \r\n"You can add walnuts and raisins to this a...   
2  \r\n"This was my grandmother's apple pie recip...   
3                              \r\n"Delish!"           
4  \r\n"Homey, sweet, custardy pie NOT like Grand...   

                                       name num_reviews nut_cals nut_carb  \
0                 Apple Pie by Grandma Ople        7720      512     67.8   
1                           Apple Crumb Pie         678      408     69.9   
2                 Grandma Ople's  Apple Pie        2166      512     67.8   
3  Grandma's Apple Pie 'Ala Mode' Moonshine         115      150       20   
4         Warm Apple Buttermilk Custard Pie         127      549     80.4   

  nut_choles nut_fat nut_prot nut_sod            rating  \
0         31    26.7      3.6     241  4.77218675613403   
1         11    14.6      3.5     139  4.69858169555664   
2         31    26.7   

In [ ]:
# Merge scraped data with API data
recipe_master=pd.merge(allrec, recipe, on='source_url', how='outer')
print recipe_master.info()

In [ ]:
# Add column for type of recipe

def recipe_type(text_in):
    try:
        if 'pie' in str(text_in).lower():
            return 'pie'
        if 'cookies' in str(text_in).lower():
            return 'cookies'
        if 'muffins' in str(text_in).lower():
            return 'muffins'
        if 'cake' in str(text_in).lower():
            return 'cake'
        if 'brownies' in str(text_in).lower():
            return 'brownies'
        if 'scones' in str(text_in).lower():
            return 'scones'
        else:
            return 'other'
    except: 
        return 0
recipe_master['recipe_type'] = recipe_master['title'].map(recipe_type)

In [140]:
# Check new columns
recipe_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1458 entries, 0 to 1457
Data columns (total 32 columns):
descrip              1441 non-null object
name                 1441 non-null object
num_reviews          1441 non-null object
nut_cals             1433 non-null object
nut_carb             1433 non-null object
nut_choles           1433 non-null object
nut_fat              1433 non-null object
nut_prot             1433 non-null object
nut_sod              1433 non-null object
rating               1441 non-null object
source_url           1458 non-null object
social_rank          1458 non-null float64
publisher            1458 non-null object
f2f_url              1458 non-null object
ingredients          1458 non-null object
recipe_id            1458 non-null object
image_url            1458 non-null object
publisher_url        1458 non-null object
title                1458 non-null object
sugar                1458 non-null float64
fat                  1458 non-null float64
dry     

In [138]:
# Drop unnecessary columns
recipe_master=recipe_master.drop(['descrip','image_url','publisher_url','f2f_url','source_url','publisher','ingredients'],axis=1)


In [143]:
# Save master to csv
recipe_master.to_csv('recipe_master1.csv', encoding='utf-8')